In [1]:
%store

Stored variables and their in-db values:
__importClassification             -> IPython.macro.Macro(u"# Handle table-like data and
__importRegression                 -> IPython.macro.Macro(u"# Handle table-like data and


In [2]:
%store -r __importRegression

In [3]:
__importRegression

/home/maviator/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
items = pd.read_csv("../input/items.csv")
categories = pd.read_csv("../input/item_categories.csv")

print (train.shape, test.shape, items.shape, categories.shape)

((2935849, 6), (214200, 3), (22170, 3), (84, 2))


In [6]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [7]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


Let's implement last month sales and check if it is comparable on error on the leaderboard
- Last month in the data is 33 (original train) and 32 (validation train)

In [8]:
last_month_sales = train.loc[train.date_block_num == 33]
last_month_sales.reset_index(inplace=True)
print last_month_sales.shape
last_month_sales.head()

(53514, 7)


,index,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2882335,23.10.2015,33,45,13315,649.0,1.0
1,2882336,05.10.2015,33,45,13880,229.0,1.0
2,2882337,02.10.2015,33,45,13881,659.0,1.0
3,2882338,12.10.2015,33,45,13881,659.0,1.0
4,2882339,04.10.2015,33,45,13923,169.0,1.0


In [9]:
last_month_sales.pop('date')
last_month_sales.pop('date_block_num')
last_month_sales.pop('item_price')
last_month_sales.pop('index')
last_month_sales.head()

,shop_id,item_id,item_cnt_day
0,45,13315,1.0
1,45,13880,1.0
2,45,13881,1.0
3,45,13881,1.0
4,45,13923,1.0


In [10]:
last_month_sales.item_cnt_day.describe()

count    53514.000000
mean         1.327802
std          9.771091
min         -1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max       2169.000000
Name: item_cnt_day, dtype: float64

In [11]:
last_month_sales = last_month_sales.groupby(['shop_id', 'item_id'], as_index=False).agg({'item_cnt_day':sum})

In [12]:
print last_month_sales.shape
last_month_sales.head()

(31531, 3)


,shop_id,item_id,item_cnt_day
0,2,31,1.0
1,2,486,3.0
2,2,787,1.0
3,2,794,1.0
4,2,968,1.0


In [13]:
last_month_sales.item_cnt_day.describe()

count    31531.000000
mean         2.253528
std         14.533295
min         -1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max       2253.000000
Name: item_cnt_day, dtype: float64

In [14]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [15]:
test_id = test.pop('ID')

In [16]:
train_labels = last_month_sales.pop('item_cnt_day')

In [17]:
lgbm = LGBMRegressor()
lgbm.fit(last_month_sales, train_labels)

LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0.0, n_estimators=10, n_jobs=-1, num_leaves=31,
       objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=50000,
       subsample_freq=1)

In [18]:
predictions = lgbm.predict(test)

In [19]:
predictions.shape

(214200,)

Clipping predictions in the [0, 20] range

In [20]:
for i in range(len(predictions)):
    if predictions[i] > 20:
        predictions[i] = 20
    if predictions[i] < 0:
        predictions[i] = 20

## Submission

In [22]:
test_submit = pd.DataFrame({'ID': test_id, 'item_cnt_month': predictions})
test_submit.head()

,ID,item_cnt_month
0,0,1.968267
1,1,1.968267
2,2,1.968267
3,3,1.968267
4,4,1.968267


In [23]:
test_submit.describe()

,ID,item_cnt_month
count,214200.000000,214200.000000
mean,107099.500000,2.093563
std,61834.358168,1.353297
min,0.000000,1.736441
25%,53549.750000,1.898515
50%,107099.500000,1.965825
75%,160649.250000,1.994430
max,214199.000000,20.000000


In [24]:
print test_submit.shape
test_submit.to_csv('lgbm.csv', index=False)
test_submit.head()

(214200, 2)


,ID,item_cnt_month
0,0,1.968267
1,1,1.968267
2,2,1.968267
3,3,1.968267
4,4,1.968267


## Notes

- Validation set is the last month in train data
- Correct validation scheme is set with RMSE calculation
- Clipping values in the [0 20] range

- Method 0: Use last month sales as predictions. RMSE:2.904, LB:1.16777
- Method 1: Train lightgbm using last month sales. RMSE: 2.73800, LB: 2.51976. Maybe last month predictions is a leak for months 33 to 34 but not 32 to 33?